In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# stdlib
import os

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

In [ ]:
# syft absolute
import syft as sy
from syft import test_settings

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

In [ ]:
high_client.worker_pools

In [ ]:
assert len(high_client.worker_pools.get_all()) == 1

In [ ]:
external_registry = test_settings.get("external_registry", default="docker.io")
external_registry

In [ ]:
result = high_client.api.services.image_registry.add(external_registry)
result

In [ ]:
image_registry_list = high_client.api.services.image_registry.get_all()
image_registry_list

In [ ]:
local_registry = image_registry_list[0]
local_registry

In [ ]:
base_worker_image = high_client.images.get_all()[0]
base_worker_image

In [ ]:
worker_dockerfile = f"""
FROM {str(base_worker_image.image_identifier)}

RUN uv pip install db-dtypes google-cloud-bigquery

""".strip()
worker_dockerfile

In [ ]:
docker_config = sy.DockerWorkerConfig(dockerfile=worker_dockerfile)
assert docker_config.dockerfile == worker_dockerfile

In [ ]:
submit_result = high_client.api.services.worker_image.submit(
    worker_config=docker_config
)
submit_result

In [ ]:
dockerfile_list = high_client.images.get_all()
dockerfile_list

In [ ]:
worker_image = next(
    (
        image
        for image in dockerfile_list
        if not image.is_prebuilt and image.config.dockerfile == worker_dockerfile
    ),
    None,
)
worker_image

In [ ]:
# worker_docker_tag = f"openmined/bigquery:{sy.__version__}"
# worker_docker_tag

In [ ]:
docker_tag = str(base_worker_image.image_identifier).replace(
    "backend", "worker-bigquery"
)
docker_tag

In [ ]:
if environment == "remote":
    docker_build_result = high_client.api.services.worker_image.build(
        image_uid=worker_image.id,
        tag=docker_tag,
        registry_uid=local_registry.id,
    )
    print(docker_build_result)

In [ ]:
if environment == "remote":
    push_result = high_client.api.services.worker_image.push(worker_image.id)
    print(push_result)

In [ ]:
docker_config = sy.PrebuiltWorkerConfig(tag=docker_tag)
docker_config

In [ ]:
result = high_client.api.services.worker_image.submit(worker_config=docker_config)
result

In [ ]:
worker_image = high_client.images.get_all()[1]
worker_image

In [ ]:
assert worker_image.is_prebuilt

In [ ]:
worker_pool_name = "bigquery-pool"
custom_pool_pod_annotations = {"bigquery-custom-pool": "Pod annotation for bigquery"}
custom_pool_pod_labels = {"bigquery-custom-pool": "Pod_label_for_bigquery"}

In [ ]:
result = high_client.api.services.worker_pool.launch(
    pool_name=worker_pool_name,
    image_uid=worker_image.id,
    num_workers=1,
    pod_annotations=custom_pool_pod_annotations,
    pod_labels=custom_pool_pod_labels,
)
result

In [ ]:
if environment == "remote":
    result = high_client.worker_pools.scale(number=2, pool_name=worker_pool_name)
    print(result)

In [ ]:
assert len(high_client.worker_pools.get_all()) == 2

In [ ]:
high_client.register(
    email="data_scientist@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="John Doe",
)

In [ ]:
high_client.settings.allow_guest_signup(enable=False)

In [ ]:
assert len(high_client.api.services.user.get_all()) == 2

In [ ]:
server.land()

In [ ]:
# !echo "$worker_dockerfile" | docker build -t $docker_tag -q -

In [ ]:
# !docker image ls | grep bigquery